In [83]:
import pandas as pd
import numpy as np
from sklearn.grid_search import RandomizedSearchCV
import xgboost
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import SGD
from sklearn.grid_search import RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasClassifier



Using Theano backend.
Using gpu device 0: GeForce GTX 745 (CNMeM is disabled, CuDNN not available)


In [2]:
df = pd.read_csv("./work_dir/boruta_filtered_train_split.csv")
y=df['y']
X=df.drop(['y'], axis=1)



In [84]:
def create_model(dropout=0.1, optimizer = 'adam'):
    # create model
    model = Sequential()
    model.add(Dense(20, input_dim=20, init='glorot_uniform', activation='relu'))
    model.add(Dense(10, init='glorot_uniform', activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(10, init='glorot_uniform', activation='relu'))
    model.add(Dense(1, init='glorot_uniform', activation='sigmoid'))
    #sgd = SGD(lr=lr, decay=1e-6, momentum=0.9, nesterov=True)
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0, nb_epoch=100, batch_size=64)


In [66]:
selected_features = pd.read_csv("./work_dir/feature_support.csv")

df_test = pd.read_csv("./work_dir/my_midterm_test_split.csv")
y_test = df_test['y']
X_test = df_test.drop('y', axis=1)
X_test = X_test.ix[:, selected_features['0'].values]

In [43]:
# Create Submission
kaggle_test = pd.read_csv("./work_dir/my_midterm_kaggle_test.csv")
kaggle_test_selected = kaggle_test.ix[:, selected_features['0'].values]  #trim to the boruta features

In [4]:
X.shape

(120000, 20)

In [5]:
y.shape

(120000,)

In [45]:
X_test.shape

(40000, 20)

In [7]:
kaggle_test_selected.shape

(40000, 20)

In [18]:
n_folds = 5

In [85]:
xgb = xgboost.XGBClassifier(nthread=6, n_estimators=300, 
                            learning_rate = 0.31, max_depth = 16, colsample_bytree = 1)
ert = ExtraTreesClassifier(n_jobs=6, random_state=42, n_estimators=500, max_features = 'log2',
                           min_samples_split = 1, max_depth =  None, criterion= 'entropy')
rfc = RandomForestClassifier(n_jobs=6, random_state=42, n_estimators=500, max_depth= None, 
                             max_features=  'auto', min_samples_split=  2, criterion=  'entropy')

clfs = [xgb, ert, rfc, model]

In [19]:
kf =StratifiedKFold(y, n_folds=n_folds, random_state=42)


In [89]:
class Blender:
    
    def __init__(self, n_folds, estimators):
        self.n_folds = n_folds
        self.estimators = estimators
        self.dataset_blend_train = None
        self.dataset_blend_test = None
        
            
    def fit_predict(self, X, y, X_submission):
        print ("Creating train and test sets for blending.")

        
        dataset_blend_train = np.zeros((X.shape[0], len(clfs)))
        dataset_blend_test = np.zeros((X_submission.shape[0], len(clfs)))
        
        skf =StratifiedKFold(y, n_folds=self.n_folds, random_state=42)


        for j, clf in enumerate(clfs):
            print(j, clf)
            dataset_blend_test_j = np.zeros((X_submission.shape[0], len(skf)))
            for i, (train, test) in enumerate(skf):
                print ("Fold", i)
                X_train = X[train]
                y_train = y[train]
                X_test = X[test]
                y_test = y[test]
                clf.fit(X_train, y_train)
                y_submission = clf.predict_proba(X_test)[:, 1]
                dataset_blend_train[test, j] = y_submission
                dataset_blend_test_j[:, i] = clf.predict_proba(X_submission)[:, 1]
            dataset_blend_test[:, j] = dataset_blend_test_j.mean(1)

        print("Blending.")
        #from sklearn.linear_model import LogisticRegression
        clf = xgboost.XGBClassifier()
        clf.fit(dataset_blend_train, y)
        y_submission = clf.predict_proba(dataset_blend_test)[:, 1]
        
        #print("Linear stretch of predictions to [0,1]")
        #y_submission = (y_submission - y_submission.min()) / (y_submission.max() - y_submission.min())
        return y_submission
    
    

In [94]:
blend = Blender(n_folds=5, estimators=clfs)

In [77]:
xgb.fit(X, y)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.31, max_delta_step=0, max_depth=16,
       min_child_weight=1, missing=None, n_estimators=300, nthread=6,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [78]:
X_test.shape

(40000, 20)

In [79]:
y_hat = xgb.predict_proba(X_test)[:,1]

In [95]:
y_submission = blend.fit_predict(X.values, y.values, X_test.values)

Creating train and test sets for blending.
0 XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.31, max_delta_step=0, max_depth=16,
       min_child_weight=1, missing=None, n_estimators=300, nthread=6,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)
Fold 0
Fold 1
Fold 2
Fold 3
Fold 4
1 ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='entropy',
           max_depth=None, max_features='log2', max_leaf_nodes=None,
           min_samples_leaf=1, min_samples_split=1,
           min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=6,
           oob_score=False, random_state=42, verbose=0, warm_start=False)
Fold 0
Fold 1
Fold 2
Fold 3
Fold 4
2 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
    

In [92]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_submission)


0.99737840091243846

In [93]:
roc_auc_score(y_test, y_hat)

0.99801121215983213